In [ ]:
# d'après https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
plt.style.use(['ggplot'])

# PARTIE 1 / Données Fictives de type linéaire pour démontrer comment fonctionne l'apprentissage

<h5> Génération de données:
Nous générons un jeu de données qui suit l'équation y = a * x + b
    ordonnée à l'origine, b =4 
    pente de la droite, a =3
    ajout de bruit gaussien afin de générer un nuage de points autour de la droite


In [ ]:
X = 2 * np.random.rand(100,1)
y = 4 +3 * X+np.random.randn(100,1)

#y = a.x + b (a= 4, b=3)

Représentons le nuage , l'ordonnée à l'origine est bien 4

In [ ]:

plt.plot(X,y,'b.')
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
_ =plt.axis([0,2,0,15])

#  Ce nuage de points suit une relation linéaire
## Il existe une résolution analytique (calcul direct) : ce modèle s'appelle la régression linéaire (simple ou multiple)
Il s'agit d'un modèle qui à partir des données (X, y) détermine a, b

In [ ]:
X_b = np.c_[np.ones((100,1)),X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(theta_best)

<h5>On est proche de nos "vrais" coefficients 4 and 3. C'est normal qu'on ne trouve pas les valeurs théoriques car on a ajotué du bruit

### Appliquons la régression avec les valeurs estimées
Prenons deux observations
Observation X1 = 0, on attend une prediction = beta_best[0] + beta_best[1]*0
Observation X2 = 2, on attend une prediction = beta_best[0] + beta_best[1]*2

In [ ]:

X_new = np.array([[0],[2]])
X_new_b = np.c_[np.ones((2,1)),X_new]
y_predict = X_new_b.dot(theta_best)
y_predict

<h5>Représentons le nuage avec la droite de régression

In [ ]:
plt.plot(X_new,y_predict,'r-')
plt.plot(X,y,'b.')
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0,2,0,15])


# Gradient Descent

<h5> Pour expliquer brièvement la descente de gradient par une analogie :
 
 imaginez que vous êtes sur une montagne et que vous êtes aveuglé et que votre tâche est de descendre de la montagne vers la terre plate sans assistance. La seule assistance que vous avez est une app qui vous indique l'altitude par rapport au niveau de la mer.
 Quelle serait votre approche? Vous commenceriez à descendre dans une direction aléatoire et demander au gadget quelle est l'altitude maintenant. Si le gadget vous indique une altitude plus élevée que votre altitude initiale, alors vous savez que vous avez commencé dans la mauvaise direction. Vous changez de direction et répétez le processus. De cette manière, après de nombreuses itérations, vous descendez enfin avec succès.

Voici une analogie pour aider à comprendre comment la descente de gradient fonctionne pour minimiser la fonction de coût dans l'apprentissage automatique.

Taille des pas dans n'importe quelle direction = Taux d'apprentissage (Learning rate)
L'app vous indique l'altitude = Fonction de coût (Loss Function)
La direction de vos pas = Gradients


## Fonction de couts & Gradients

<h5> Ci dessous les équations pour calculer la fonction de cout et les gradients pour le modèle de régression linéaire.



<b>Cost</b>
\begin{equation}
J(\theta) = 1/2m \sum_{i=1}^{m} (h(\theta)^{(i)} - y^{(i)})^2 
\end{equation}

<b>Gradient</b>

\begin{equation}
\frac{\partial J(\theta)}{\partial \theta_j} = 1/m\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_j^{(i)}
\end{equation}

<b>Gradients</b>
\begin{equation}
\theta_0: = \theta_0 -\alpha . (1/m .\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_0^{(i)})
\end{equation}
\begin{equation}
\theta_1: = \theta_1 -\alpha . (1/m .\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_1^{(i)})
\end{equation}
\begin{equation}
\theta_2: = \theta_2 -\alpha . (1/m .\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_2^{(i)})
\end{equation}

\begin{equation}
\theta_j: = \theta_j -\alpha . (1/m .\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_0^{(i)})
\end{equation}

In [ ]:

def  cal_cost(theta,X,y):
    '''
    
   Calculer fonction de couts
    Exemple sur un vecteur à une dimension X
    theta = Vector of thetas (coefficients de la régression linéaire)
    X     = Row of X's np.zeros((2,j))
    y     = Actual y's np.zeros((2,1))
    
    where:
        j is the no of features
    '''
    
    m = len(y)
    
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    return cost


In [ ]:
# fonction de descente de gradient, ajout d'un parametre pour afficher par iter
def gradient_descent(X,y,theta,learning_rate=0.01,iterations=100,printiter=True):
    '''
    X    = Matrix of X with added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    # vecteur contenant autant d'éléments que d'itérations avec la fonction de cout qui décroit
    cost_history = np.zeros(iterations)
    # vecteur 2D qui contient l'évolution des deux parametres 
    theta_history = np.zeros((iterations,2))
    
    # itérations
    for it in range(iterations):
        # on calcule la prédiction selon le vecteur de coef
        prediction = np.dot(X,theta)
        
        # actualisation des coef en fonction du learning rate et du gradient du param
        theta = theta -(1/m)*learning_rate*( X.T.dot((prediction - y)))
        # mise à jour de l'historique des coef
        theta_history[it,:] =theta.T
        # mise à jour de la fonction de cout
        cost_history[it]  = cal_cost(theta,X,y)
        if printiter==True:
            # on affiche itération, coef à deux dec et la fonction de cout
            if it<=5:
                print('itération :{}, vecteur des coef :{:0.2f}-{:0.2f}, fonction de perte: {:0.1f}'.format(it,theta[0][0],theta[1][0],cal_cost(theta,X,y)))
            else:
                if it%25==0:
                    print('dérivé du coef de x',(1/m)*learning_rate*( X.T.dot((prediction - y)))[1][0])
                    print('itération :{}, vecteur des coef :{:0.2f}-{:0.2f}, fonction de perte: {:0.1f}'.format(it,theta[0][0],theta[1][0],cal_cost(theta,X,y)))
                else:
                    pass
        
    return theta, cost_history, theta_history

    

<h3> Réalisons une descente de gradient avec 1000 iterations et learning rate 0.01. 
Le vecteur de coef. démarre avec un tirage aléatoire (gaussien)

In [ ]:
lr =0.1
n_iter = 501

theta = np.random.randn(2,1)

# matrice X avec une constante
X_b = np.c_[np.ones((len(X),1)),X]
theta,cost_history,theta_history = gradient_descent(X_b,y,theta,lr,n_iter)


print('Theta0:          {:0.3f},\nTheta1:          {:0.3f}'.format(theta[0][0],theta[1][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

<h3> Let's plot the cost history over iterations

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

ax.set_ylabel('J(Theta)')
ax.set_xlabel('Iterations')
_=ax.plot(range(n_iter),cost_history,'b.')

<h3> Apres 150 iterations, le cout devient "flat" : les itérations suivantes apportent plus rien. Concentrons nous sur les 200 premières itérations

In [ ]:

fig,ax = plt.subplots(figsize=(10,8))
_=ax.plot(range(200),cost_history[:200],'b.')

<b>Les couts descendent très vite dans les premières itérations puis plafonne

### Explorons différentes combinaisons : itérations x learning rate

### Fonction pour représenter la descente de gradient

In [ ]:

def plot_GD(n_iter,lr,ax,ax1=None):
     """
     n_iter = no of iterations
     lr = Learning Rate
     ax = Axis to plot the Gradient Descent
     ax1 = Axis to plot cost_history vs Iterations plot

     """
     _ = ax.plot(X,y,'b.')
     theta = np.random.randn(2,1)

     tr =0.1
     cost_history = np.zeros(n_iter)
     for i in range(n_iter):
        pred_prev = X_b.dot(theta)
        theta,h,_ = gradient_descent(X_b,y,theta,lr,1)
        pred = X_b.dot(theta)

        cost_history[i] = h[0]

        if ((i % 25 == 0) ):
            _ = ax.plot(X,pred,'r-',alpha=tr)
            if tr < 0.8:
                tr = tr+0.2
     if not ax1== None:
        _ = ax1.plot(range(n_iter),cost_history,'b.')  

### Représentation selon différentes combinaisons : itérations x learning rate

In [ ]:
fig = plt.figure(figsize=(30,25),dpi=200)
fig.subplots_adjust(hspace=0.4, wspace=0.4)

it_lr =[(2000,0.001),(500,0.01),(200,0.05),(100,0.1)]
count =0
for n_iter, lr in it_lr:
    count += 1
    
    ax = fig.add_subplot(4, 2, count)
    count += 1
   
    ax1 = fig.add_subplot(4,2,count)
    
    ax.set_title("lr:{}".format(lr))
    ax1.set_title("Iterations:{}".format(n_iter))
    plot_GD(n_iter,lr,ax,ax1)
    

<b> Les lignes rouges montrent comment la descente de gradient progresse lentement vers les valeurs finales

## Représentations par cas

In [ ]:
_,ax = plt.subplots(figsize=(14,10))
plot_GD(100,0.1,ax)

# Stochastic Gradient Descent

In [ ]:
def stochastic_gradient_descent(X,y,theta,learning_rate=0.01,iterations=10):
    '''
    X    = Matrix of X with added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    cost_history = np.zeros(iterations)
    
    
    for it in range(iterations):
        cost =0.0
        # on actualise par individu i et on passe tous les individus
        for i in range(m):
            rand_ind = np.random.randint(0,m)
            X_i = X[rand_ind,:].reshape(1,X.shape[1])
            y_i = y[rand_ind].reshape(1,1)
            prediction = np.dot(X_i,theta)

            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost
        
    return theta, cost_history

In [ ]:
lr =0.5
n_iter = 50

theta = np.random.randn(2,1)

X_b = np.c_[np.ones((len(X),1)),X]
theta,cost_history = stochastic_gradient_descent(X_b,y,theta,lr,n_iter)


print('Theta0:          {:0.3f},\nTheta1:          {:0.3f}'.format(theta[0][0],theta[1][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))

ax.set_ylabel('{J(Theta)}',rotation=0)
ax.set_xlabel('{Iterations}')
theta = np.random.randn(2,1)

_=ax.plot(range(n_iter),cost_history,'b.')

# Mini Batch Gradient Descent

In [ ]:
def minibatch_gradient_descent(X,y,theta,learning_rate=0.01,iterations=10,batch_size =20):
    '''
    X    = Matrix of X without added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)
    
    for it in range(iterations):
        cost =0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
         # on actualise par i paquet d'individu (batch size) et tout le paquet est utilisé pour actualiser l'estimation
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]
            
            X_i = np.c_[np.ones(len(X_i)),X_i]
           
            prediction = np.dot(X_i,theta)

            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost
        
    return theta, cost_history

In [ ]:
lr =0.1
n_iter = 200

theta = np.random.randn(2,1)


theta,cost_history = minibatch_gradient_descent(X,y,theta,lr,n_iter)


print('Theta0:          {:0.3f},\nTheta1:          {:0.3f}'.format(theta[0][0],theta[1][0]))
print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))

ax.set_ylabel('{J(Theta)}',rotation=0)
ax.set_xlabel('{Iterations}')
theta = np.random.randn(2,1)

_=ax.plot(range(n_iter),cost_history,'b.')